# Baseline script of San Francisco Crime Classification

Baseline script. Hope this helps yours.

In [129]:
import numpy as np
import pandas as pd

## Load Data

In [74]:
train = pd.read_pickle("../all_df.p")
train.head(3)

,날짜,순위,영화명,개봉일,매출액,매출액 점유율,매출액증감 (전일대비),매출액증감율 (전일대비),누적매출액,관객수,...,스크린수,상영횟수,대표국적,국적,제작사,배급사,등급,장르,감독,배우
4,2014년 08월 07일(목),1,명량,2014-07-30 00:00:00,4892511700,0.595,-380933700,-0.1,61033902816,652776,...,1239,6776,한국,한국,(주)빅스톤픽쳐스,씨제이이앤엠(주),15세이상관람가,"사극,액션",김한민,"최민식,류승룡,조진웅,김명곤,진구,이정현,권율,노민우,김태훈,이승준,오타니 료헤이,..."
5,2014년 08월 07일(목),2,해적: 바다로 간 산적,2014-08-06 00:00:00,1890503200,0.23,-130889400,-0.1,4137374300,254862,...,795,3932,한국,한국,(주)하리마오픽쳐스,롯데쇼핑㈜롯데엔터테인먼트,12세이상관람가,어드벤처,이석훈,"김남길,손예진,유해진,이경영,오달수,김태우,박철민,신정근,김원해,조달환,조희봉,정성..."
6,2014년 08월 07일(목),3,드래곤 길들이기 2,2014-07-23 00:00:00,437352300,0.053,-27892900,-0.1,19775433889,58182,...,423,1016,미국,미국,드림웍스 애니메이션,씨제이이앤엠(주),전체관람가,"애니메이션,액션,어드벤처",딘 데블로이스,"제이 바루첼,제라드 버틀러,케이트 블란쳇,아메리카 페레라"


In [75]:
import re

train["날짜"] = train["날짜"].apply(lambda date: re.sub('[(년월화수목금토일)]', '', date))
train["날짜"] = train["날짜"].apply(lambda date: date.replace(" ", "-"))
train["날짜"].head(3)

train["개봉일"] = pd.to_datetime(train["개봉일"], errors='coerce')
train["개봉일"].head()

train["상영기간"] = pd.to_datetime(train["날짜"]) - pd.to_datetime(train["개봉일"])
train = train.dropna(subset=['상영기간'])

train = train[train["상영기간"] >= '0 days']
train = train[train["상영기간"] < '30 days']

In [76]:
train.shape

(28586, 24)

In [77]:
test = pd.read_pickle("../test.p")

In [78]:
test

,날짜,영화명,개봉일,대표국적,국적,제작사,배급사,등급,장르,감독,배우
0,2016년 09월 30일(금),"고산자, 대동여지도",2016-09-07,한국,한국,(주)시네마서비스,씨제이이앤엠(주),전체관람가,사극,강우석,"차승원,유준상,김인권,남지현,신동미,남경읍,태인호,공형진,김종수"
1,2016년 09월 30일(금),매그니피센트 7,2016-09-14,미국,미국,,유니버설픽쳐스인터내셔널 코리아(유),15세이상관람가,"액션, 서부극(웨스턴)",안톤 후쿠아,"덴젤 워싱턴,크리스 프랫,이병헌,에단 호크,맷 보머,빈센트 도노프리오,해일리 베넷,..."
2,2016년 09월 30일(금),카페 소사이어티,2016-09-14,미국,미국,,CGV아트하우스,전체관람가,"멜로/로맨스, 드라마",우디 앨런,"제시 아이젠버그, 크리스틴 스튜어트, 스티브 카렐, 블레이크 라이블리"


In [79]:
vector_columns = test.columns.drop(['날짜', '영화명', '개봉일'])

In [80]:
vector_columns

Index(['대표국적', '국적', '제작사', '배급사', '등급', '장르', '감독', '배우'], dtype='object')

In [81]:
d_list = []
for each in train.iterrows():
    row = ""
    for column in vector_columns.values :
        if type(each[1][column]) == str :
            row = row + each[1][column] + " "
    d_list.append(row)

In [82]:
y_list = train["누적관객수"].values

In [83]:
test_d_list = []
for each in test.iterrows():
    row = ""
    for column in vector_columns.values :
        if type(each[1][column]) == str :
            row = row + each[1][column] + " "
    test_d_list.append(row)

In [84]:
test_d_list

['한국 한국 (주)시네마서비스 씨제이이앤엠(주) 전체관람가 사극 강우석 차승원,유준상,김인권,남지현,신동미,남경읍,태인호,공형진,김종수 ',
 '미국 미국  유니버설픽쳐스인터내셔널 코리아(유) 15세이상관람가 액션, 서부극(웨스턴) 안톤 후쿠아 덴젤 워싱턴,크리스 프랫,이병헌,에단 호크,맷 보머,빈센트 도노프리오,해일리 베넷,캠 지갠뎃 ',
 '미국 미국  CGV아트하우스 전체관람가 멜로/로맨스, 드라마 우디 앨런 제시 아이젠버그, 크리스틴 스튜어트, 스티브 카렐, 블레이크 라이블리 ']

## Vectorize


In [85]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [86]:
vectorizer = TfidfVectorizer()
x_list = vectorizer.fit_transform(d_list)

In [87]:
x_list

<28586x9147 sparse matrix of type '<class 'numpy.float64'>'
	with 416598 stored elements in Compressed Sparse Row format>

In [88]:
x_list.shape

(28586, 9147)

In [89]:
x_list._shape = (x_list.shape[0], x_list.shape[1] + 1)

In [90]:
train["상영기간"] = train["상영기간"].astype('timedelta64[D]').astype(int)

In [91]:
train.iloc[0]['상영기간']

8

In [92]:
x_list.shape[0]

28586

In [93]:
train.shape

(28586, 24)

In [94]:
days = train['상영기간'].values

In [95]:
for i in range(x_list.shape[0]) :
    x_list[i, x_list.shape[1] - 1] = days[i]

C:\Users\Yoo\Anaconda3\lib\site-packages\scipy\sparse\compressed.py:730: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


## CV

In [ ]:
train['누적관객수']

In [99]:
def RMSE(y, ypred):
    rmse = np.sqrt(np.mean((y - ypred) ** 2))
    return rmse

In [118]:

from sklearn.tree import DecisionTreeRegressor
model_rfr = DecisionTreeRegressor()
model_rfr.fit(x_list, y_list)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, presort=False, random_state=None,
           splitter='best')

In [102]:

from sklearn.ensemble import RandomForestRegressor
model_rfr = RandomForestRegressor(n_estimators=100, n_jobs=-1)
model_rfr.fit(x_list, y_list)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=100, n_jobs=-1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [119]:
predict_values = model_rfr.predict(x_list)

In [120]:
error_rfr = RMSE(y_list, predict_values)

In [121]:
error_rfr

911.35851495026543

In [122]:
test

,날짜,영화명,개봉일,대표국적,국적,제작사,배급사,등급,장르,감독,배우,상영기간
0,2016-09-30,"고산자, 대동여지도",2016-09-07,한국,한국,(주)시네마서비스,씨제이이앤엠(주),전체관람가,사극,강우석,"차승원,유준상,김인권,남지현,신동미,남경읍,태인호,공형진,김종수",23
1,2016-09-30,매그니피센트 7,2016-09-14,미국,미국,,유니버설픽쳐스인터내셔널 코리아(유),15세이상관람가,"액션, 서부극(웨스턴)",안톤 후쿠아,"덴젤 워싱턴,크리스 프랫,이병헌,에단 호크,맷 보머,빈센트 도노프리오,해일리 베넷,...",16
2,2016-09-30,카페 소사이어티,2016-09-14,미국,미국,,CGV아트하우스,전체관람가,"멜로/로맨스, 드라마",우디 앨런,"제시 아이젠버그, 크리스틴 스튜어트, 스티브 카렐, 블레이크 라이블리",16


In [123]:
vectorizer.vocabulary_

{'타이거시네마': 7935,
 '샬롯': 3483,
 '이이자와': 5941,
 '임정예': 6226,
 '장민애': 6313,
 '캘링': 7484,
 '영화사봄': 4895,
 '이영아': 5860,
 '장원형': 6337,
 '탁원제': 8000,
 '아야네': 4314,
 '쿠퍼': 7743,
 '김민수': 579,
 '카메론': 7339,
 'new': 31,
 '번스타인': 3019,
 '테넌트': 8023,
 '맥도넬': 2199,
 '제이콥': 6693,
 '하라': 8606,
 '부케': 3180,
 '까뜨린느': 971,
 '김지영': 847,
 '박배일': 2711,
 '박용범': 2777,
 '김가현': 465,
 '후지오카': 9059,
 '카펜터': 7443,
 '스티어스': 3974,
 '셀마': 3638,
 '황찬성': 9035,
 '타란티노': 7920,
 '루카': 1841,
 '그라함': 393,
 '지로': 6951,
 '로벤': 1741,
 '김라윤': 548,
 '이철민': 6075,
 '키린': 7860,
 '강동석': 105,
 '김영필': 715,
 '에바': 4743,
 '박진영': 2854,
 '글레이저': 435,
 '엄효섭': 4700,
 '이원희': 5908,
 '허브너': 8808,
 '이재석': 5969,
 '메싱': 2280,
 '도로신스키': 1398,
 '놀테': 1182,
 '장현희': 6380,
 '박영수': 2769,
 '김현희': 927,
 '코스모': 7602,
 '토비아스': 8089,
 '로우': 1762,
 '맥당걸': 2193,
 '김근배': 491,
 '프릴루흐니': 8490,
 '이치하라': 6087,
 '신원호': 4106,
 '최상훈': 7137,
 '플로렌틴': 8505,
 '씨네힐': 4180,
 '주현': 6915,
 'the': 43,
 '장희철': 6385,
 '이진욱': 6060,
 '이윤기': 5918,
 '세르지오': 3603,
 '아르만도': 4228,
 '

In [124]:
test_x_list = vectorizer.transform(test_d_list)

In [125]:
test_x_list._shape = (3, 9148)

In [126]:
import re

test["날짜"] = test["날짜"].apply(lambda date: re.sub('[(년월화수목금토일)]', '', date))
test["날짜"] = test["날짜"].apply(lambda date: date.replace(" ", "-"))
test["날짜"].head(3)

test["개봉일"] = pd.to_datetime(test["개봉일"], errors='coerce')
test["개봉일"].head()

test["상영기간"] = pd.to_datetime(test["날짜"]) - pd.to_datetime(test["개봉일"])
test = test.dropna(subset=['상영기간'])

test = test[test["상영기간"] >= '0 days']
test = test[test["상영기간"] < '30 days']
test["상영기간"] = test["상영기간"].astype('timedelta64[D]').astype(int)
test_days = test['상영기간'].values

In [127]:
for i in range(test_x_list.shape[0]) :
    test_x_list[i, test_x_list.shape[1] - 1] = test_days[i]

C:\Users\Yoo\Anaconda3\lib\site-packages\scipy\sparse\compressed.py:730: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


In [128]:
model_rfr.predict(test_x_list)

array([  141054.,  2624759.,     4030.])